## 1. Carga e Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.utils import class_weight

Cargamos los datos para el entrenamiento del mejor modelo encontrado previamente.

In [8]:
df_train = pd.read_csv('./salario.csv', na_values=['?'], skipinitialspace=True)
X_test_raw = pd.read_csv('./salario_test.csv', na_values=['?'], skipinitialspace=True)

In [9]:
mapa_salario = {'<=50K': 0, '>50K': 1}
df_train['salario'] = df_train['salario'].map(mapa_salario)

target_col = 'salario'
y_train = df_train[target_col]
X_train_raw = df_train.drop(target_col, axis=1)

Eliminamos la columna ID de X_test_raw antes de realizar el preprocesado

In [ ]:
submission_ids = X_test_raw['ID']
X_test_raw = X_test_raw.drop('ID', axis=1)
submission_ids = X_test_raw.index

Imputamos los valores nulos con la moda de cada columna como en el preprocesado original

In [ ]:
columnas_con_nulos = ['trabajo', 'trabajo.1', 'pais-origen']
imputers = {}
for col in columnas_con_nulos:
    moda = X_train_raw[col].mode()[0]
    imputers[col] = moda
    X_train_raw[col] = X_train_raw[col].fillna(moda)
    X_test_raw[col] = X_test_raw[col].fillna(moda)
print("Nulos imputados.")

Nulos imputados.


Utilizamos el pipeline de preprocesado v2 como en el preprocesado original

In [ ]:
numeric_features = ['edad', 'ganancias_inversiones', 'perdidas_inversiones', 'horas-trabajo_semana']
ordinal_features = ['estudios']
binary_features = ['sexo']
nominal_features = ['trabajo', 'estado-civil', 'trabajo.1', 'posicion-familiar', 'etnia', 'pais-origen']
education_order = [
    'Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', 
    '10th', '11th', '12th', 'HS-grad', 'Some-college', 
    'Assoc-voc', 'Assoc-acdm', 'Bachelors', 'Masters', 
    'Prof-school', 'Doctorate'
]
numeric_transformer_v2 = StandardScaler()
ordinal_transformer = OrdinalEncoder(
    categories=[education_order],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)
binary_transformer = OrdinalEncoder()
nominal_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor_v2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_v2, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('bin', binary_transformer, binary_features),
        ('nom', nominal_transformer, nominal_features)
    ],
    remainder='passthrough'
)

In [14]:
X_train_processed = preprocessor_v2.fit_transform(X_train_raw)
X_test_processed = preprocessor_v2.transform(X_test_raw)

Seleccionamos las características de Consenso V2 como en el preprocesado original

In [16]:
k_best_selector = SelectKBest(score_func=f_classif, k=30)
k_best_selector.fit(X_train_processed, y_train)
features_elegidas_filter = k_best_selector.get_support()
estimator_rfe = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42, class_weight='balanced')
rfe_selector = RFE(estimator=estimator_rfe, n_features_to_select=30, step=1)
rfe_selector.fit(X_train_processed, y_train)
features_elegidas_wrapper = rfe_selector.support_
lasso_model = LogisticRegression(
    penalty='l1', solver='liblinear', C=0.1, 
    random_state=42, max_iter=1000, class_weight='balanced'
)
embedded_selector = SelectFromModel(
    lasso_model, max_features=30, threshold=-np.inf
) 
embedded_selector.fit(X_train_processed, y_train)
features_elegidas_embedded = embedded_selector.get_support()
features_consenso_final = (features_elegidas_filter & features_elegidas_wrapper) | \
                          (features_elegidas_filter & features_elegidas_embedded) | \
                          (features_elegidas_wrapper & features_elegidas_embedded)
num_features_final = np.sum(features_consenso_final)
print(f"Número de características seleccionadas por consenso: {num_features_final}")

Número de características seleccionadas por consenso: 24


In [18]:
X_train = X_train_processed[:, features_consenso_final]
X_test = X_test_processed[:, features_consenso_final]
print(f"X_train listo con shape: {X_train.shape}")
print(f"y_train listo con shape: {y_train.shape}")
print(f"X_test listo con shape: {X_test.shape}")

X_train listo con shape: (27998, 24)
y_train listo con shape: (27998,)
X_test listo con shape: (4563, 24)


Para escoger el mejor modelo para la práctica hemos tenido un debate:
- Nosotros hemos considerado darle más importancia al F1 score obtenido de la clase 1, siendo el modelo de XgBoost el que mejor resultado nos ha dado en este aspecto, con un F1 score de 0.7230.
- Sin embargo, hemos decidido finalmente utilizar el modelo de Keras, porque en la competición de Kaggle es el que mejor resultado nos ha dado, siendo el F1 score de 0.861.

Generamos el modelo de Keras con los mejores hiperparámetros encontrados en el supervised.ipynb

In [ ]:
def create_keras_model(meta, dropout_rate=0.3):
    n_features_in = meta["n_features_in_"]
    model = Sequential([
        Input(shape=(n_features_in,)),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(32, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy'
    )
    return model

class_weights = class_weight.compute_class_weight('balanced',classes=np.unique(y_train),y=y_train)
keras_class_weights = dict(enumerate(class_weights))
print(f"Pesos de clase para Keras: {keras_class_weights}")

final_keras = KerasClassifier(
    model=create_keras_model,
    dropout_rate=0.3,
    epochs=50,
    batch_size=64,
    fit__class_weight=keras_class_weights,
    verbose=1
)
final_keras.fit(X_train, y_train)

Generamos las predicciones con el modelo de Keras dando como resultado un archivo CSV con las predicciones, como nos piden en la práctica.

In [ ]:
y_pred_proba_keras = final_keras.predict_proba(X_test)
predictions_keras = (y_pred_proba_keras[:, 1] > 0.69).astype(int)

labels_keras = np.vectorize(mapa_salario.get)(predictions_keras)

submission_keras = pd.DataFrame({
    'ID': submission_ids,
    'Salario': labels_keras
})
submission_keras.to_csv('test_labels.csv', index=False)
print(submission_keras.head())